In [ ]:
import os
import quantities as pq
import numpy as np
import math 
import neo
import json
from pathlib import Path
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, Cursor
from scipy.interpolate import interp2d
from scipy.signal import find_peaks
from scipy.stats import zscore
import pickle
import os
from scipy.interpolate import griddata
import logging
import sys 
import shutil
from bisect import bisect_left
from ast import literal_eval
import seaborn as sns
from itertools import groupby
from ephyviewer import mkQApp, MainViewer, TraceViewer
from IPython.display import display
from ipyfilechooser import FileChooser
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path=f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/AB_GlobalAnalysis/AVG_Osc_2024-10-05_10_30_36_corrScor/Baseline/AllUnits/Spdl/L1_PSTH.xlsx"
dataPSTH=pd.read_excel(path, header=None, index_col=0, sheet_name=None)

In [ ]:
data.keys()

In [ ]:
data['CaBSL_AllNrS1'].index

In [ ]:
plt.close()
plt.plot(data['CaBSL_AllNrS1'].loc['BlackLinesOK10'])
plt.show()

In [ ]:
Osc='Spdl'
NrSubtype='L1'
mice='GreenDotsOK'
mice='RedLinesOK'
mice='BlackLinesOK'

path=f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis/Osc_2024_10_05_04_01_10_AH_correctScor/{Osc}_CaPSTH_CoupledS1Baseline_{mice}.pkl"
with open(path, 'rb') as file:
    dict_All_ActivityCa_SPDLS1_Baseline=pickle.load(file)
path=f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis/Osc_2024_10_05_04_01_10_AH_correctScor/{Osc}_CaPSTH_CoupledPFCBaseline_{mice}.pkl"
with open(path, 'rb') as file:
    dict_All_ActivityCa_SPDLPFC_Baseline=pickle.load(file)
path=f"//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/Baseline_recording_ABmodified/AB_Analysis/Osc_2024_10_05_04_01_10_AH_correctScor/{Osc}_CaPSTH_CoupledS1PFCBaseline_{mice}.pkl"
with open(path, 'rb') as file:
    dict_All_ActivityCa_SPDLS1PFC_Baseline=pickle.load(file)

In [ ]:
dict_All_ActivityCa_SPDLS1_Baseline.keys()

In [ ]:
nr='ThreeColDotsOK41'
nr='RedLinesOK4'
nr='ThreeColDotsOK335'
nr='ThreeColDotsOK1346'
nr='RedLinesOK61'
nr='RedLinesOK61'
nr='BlackLinesOK0'


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def process_and_plot(data, ax, vmin=0, vmax=1):
    #data1=data.iloc[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    data1=data[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    n_cols = np.shape(data1)[1] // 4  # Floor division to get the number of columns in 1/4th
    baseline_columns = data1[:, :n_cols]  # Select the first 'n_cols' columns
    mean_baseline = np.nanmean(baseline_columns,axis=1)
    data2 = data1-mean_baseline[:, np.newaxis]
    sns.heatmap(data2, ax=ax, cmap='viridis', xticklabels=False, yticklabels=False, vmin=vmin, vmax=vmax)
plt.close()

fig, axes = plt.subplots(1, 3, figsize=(6, 2))

# Check if subplots are correctly created
process_and_plot(dict_All_ActivityCa_SPDLS1_Baseline[nr], axes[0])
process_and_plot(dict_All_ActivityCa_SPDLPFC_Baseline[nr], axes[1])
process_and_plot(dict_All_ActivityCa_SPDLS1PFC_Baseline[nr], axes[2])

plt.savefig(f'C:/Users/Manip2/Documents/ElifePaper/Rawdata/Extract_{nr}_heatmap_Spdl_Catraces.svg', format='svg', bbox_inches='tight', pad_inches=0, transparent=True)
plt.show()


In [ ]:
plt.close()
fig, axes = plt.subplots(1, 3, figsize=(4, 2))  # Increased figure size for better readability

cmap = sns.light_palette("#008B8B", as_cmap=True)
#cmap = sns.light_palette("black", as_cmap=True)

def plot_lines(data, ax, title, xlabel=None, title_color='black', cmap=cmap):
    #data1=data.iloc[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    data1=data[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    n_cols = np.shape(data1)[1]// 4  # Floor division to get the number of columns in 1/4th
    baseline_columns = data1[:, :n_cols]  # Select the first 'n_cols' columns
    mean_baseline = np.nanmean(baseline_columns,axis=1)
    data2 = data1-mean_baseline[:, np.newaxis]
    data2=np.array(data2)
    num_lines = data2.shape[0]
    colors = cmap(np.linspace(0., .8, num_lines))
    ax.text(-1,0, f'#1', color='black', ha='right', va='center', fontsize=8) 
    # Plot each line with a label
    for i in range(num_lines):
        line_data = data2[i] - i * 1.5
        ax.plot(0, min(line_data), line_data, color=colors[i], linewidth=1)
    ax.text(-1, min(line_data), f'#{i+1}', color='black', ha='right', va='center', fontsize=8) 
    ax.set_title(title, fontsize=12, color=title_color, pad=10)  # Added padding for the title
    if xlabel:
        ax.set_xlabel(xlabel, labelpad=2, fontsize=10)    
    ax.axis('off')  # Hide axis lines and labels

data = dict_All_ActivityCa_SPDLS1_Baseline[nr][::]*1   #[::2] Taking every other row
plot_lines(data, axes[0], 'S1 only', title_color='rebeccapurple')
data = dict_All_ActivityCa_SPDLPFC_Baseline[nr][::]*1
plot_lines(data, axes[1], 'PFC only', title_color='mediumseagreen')
data = dict_All_ActivityCa_SPDLS1PFC_Baseline[nr][::]*1
plot_lines(data, axes[2], 'S1&PFC')

output_path = f'C:/Users/Manip2/Documents/ElifePaper/Rawdata/Extract_line_{nr}_Spdl_CatracesLag.svg'
plt.savefig(output_path, format='svg', bbox_inches='tight', pad_inches=0, transparent=True)

# Show the plot
plt.show()


In [ ]:
plt.close()
fig, axes = plt.subplots(1, 3, figsize=(4, 10.5))  # Increased figure size for better readability

#cmap = sns.light_palette("#008B8B", as_cmap=True)
cmap = sns.light_palette("black", as_cmap=True)

def plot_lines(data, ax,title_color='black', cmap=cmap):
    data1=data[:, data.shape[1] // 10 *4  : data.shape[1] // 10 * 6]
    n_cols = np.shape(data1)[1]// 4  # Floor division to get the number of columns in 1/4th
    baseline_columns = data1[:, :n_cols]   # Select the first 'n_cols' columns
    mean_baseline = np.nanmean(baseline_columns,axis=1)
    data2 = data1-mean_baseline[:, np.newaxis]
    ax.plot(np.nanmean(data2, axis=0), color=title_color, linewidth=2)
    ax.set_ylim(-.6, 1)    
    ax.axis('off')  # Hide axis lines and labels

data = dict_All_ActivityCa_SPDLS1_Baseline[nr]*100 # [::2] Taking every other row
plot_lines(data, axes[0],  title_color='rebeccapurple')
data = dict_All_ActivityCa_SPDLPFC_Baseline[nr]*100
plot_lines(data, axes[1], title_color='mediumseagreen')
data = dict_All_ActivityCa_SPDLS1PFC_Baseline[nr]*100
plot_lines(data, axes[2])

output_path = f'C:/Users/Manip2/Documents/ElifePaper/Rawdata/Extract_AVG_{nr}_Spdl_CatracesLag.svg'
plt.savefig(output_path, format='svg', bbox_inches='tight', pad_inches=0, transparent=True)

# Show the plot
plt.show()


In [ ]:
Nr='L1'
#Nr='L2&3'

if Nr=='L1':
    path="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/AB_GlobalAnalysis/AVG_Osc_2024-10-05_10_30_36_corrScor/L1_Spdl_Global.xlsx"
    dfa=pd.read_excel(path)

    path="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/AB_GlobalAnalysis/AVG_VigSt_2024-10-03_16_09_22_AB_newScor/Baseline/All/L1_VigSt_nAUC.xlsx"
    dfi=pd.read_excel(path)
else:
    path="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/AB_GlobalAnalysis/AVG_Osc_2024-10-05_10_30_36_corrScor/L2&3_Spdl_Global.xlsx"
    dfa=pd.read_excel(path)

    path="//10.69.168.1/crnldata/waking/audrey_hay/L1imaging/AnalysedMarch2023/Gaelle/AB_GlobalAnalysis/AVG_VigSt_2024-10-03_16_09_22_AB_newScor/Baseline/All/L2&3_VigSt_nAUC.xlsx"
    dfi=pd.read_excel(path)


dfii = dfi.pivot_table(index='Unit_ID', values='NREM', fill_value=0)
dfii2 = dfi.pivot_table(index='Unit_ID', values='RatioNREM_REM', fill_value=0)
dfAll=pd.concat([dfii,dfii2], axis=1, ignore_index=False)


dfd = dfa[dfa['GlobalSpindle'] == 'Local'] 
dfd = dfd[dfd['SpdlStartLocation'] == 'S1'] 
dfd['NormAUCBeforeS1']=dfd['AUC_calciumBefore']/(dfd['SpdlDuration']/1000)
dff = dfd.pivot_table(index='Unit_ID', values='NormAUCBeforeS1', aggfunc='mean', fill_value=0)
dfAll=pd.concat([dfAll,dff], axis=1, ignore_index=False)

dfd['NormAUCDuringS1']=dfd['AUC_calciumDuring']/(dfd['SpdlDuration']/1000)
dff = dfd.pivot_table(index='Unit_ID', values='NormAUCDuringS1', aggfunc='mean', fill_value=0)
dfAll=pd.concat([dfAll,dff], axis=1, ignore_index=False)



dfd = dfa[dfa['GlobalSpindle'] == 'Local'] 
dfd = dfd[dfd['SpdlStartLocation'] == 'PFC'] 
dfd['NormAUCBeforePFC']=dfd['AUC_calciumBefore']/(dfd['SpdlDuration']/1000)
dff = dfd.pivot_table(index='Unit_ID', values='NormAUCBeforePFC', aggfunc='mean', fill_value=0)
dfAll=pd.concat([dfAll,dff], axis=1, ignore_index=False)

dfd['NormAUCDuringPFC']=dfd['AUC_calciumDuring']/(dfd['SpdlDuration']/1000)
dff = dfd.pivot_table(index='Unit_ID', values='NormAUCDuringPFC', aggfunc='mean', fill_value=0)

dfAll=pd.concat([dfAll,dff], axis=1, ignore_index=False)

dfd = dfa[dfa['Drug'] == 'Baseline'] 
dfd = dfd[dfd['GlobalSpindle'] == 'Global'] 
dfd['NormAUCBeforeGBL']=dfd['AUC_calciumBefore']/(dfd['SpdlDuration']/1000)
dff = dfd.pivot_table(index='Unit_ID', values='NormAUCBeforeGBL', aggfunc='mean', fill_value=0)
dfAll=pd.concat([dfAll,dff], axis=1, ignore_index=False)

dfd['NormAUCDuringGBL']=dfd['AUC_calciumDuring']/(dfd['SpdlDuration']/1000)
dff = dfd.pivot_table(index='Unit_ID', values='NormAUCDuringGBL', aggfunc='mean', fill_value=0)

dfAll=pd.concat([dfAll,dff], axis=1, ignore_index=False)

dfAll['S1diff']=dfAll['NormAUCDuringS1']-dfAll['NormAUCBeforeS1']
dfAll['PFCdiff']=dfAll['NormAUCDuringPFC']-dfAll['NormAUCBeforePFC']
dfAll['GBLdiff']=dfAll['NormAUCDuringGBL']-dfAll['NormAUCBeforeGBL']

dfAll = dfAll.dropna(subset=['RatioNREM_REM'])

df_cleaned=dfAll.copy()
df_cleaned = df_cleaned.dropna(subset=['S1diff'])
dfAllnrem=df_cleaned[df_cleaned['RatioNREM_REM']>0.5]
dfAllrem=df_cleaned[df_cleaned['RatioNREM_REM']<-0.5]
dfAllns=df_cleaned[df_cleaned['RatioNREM_REM']<=0.5]
dfAllns=dfAllns[dfAllns['RatioNREM_REM']>=-0.5]

print('ACTIVE')

print('S1')
print(sum(dfAllnrem['S1diff']>0)/len(df_cleaned)*100)
print(sum(dfAllns['S1diff']>0)/len(df_cleaned)*100)
print(sum(dfAllrem['S1diff']>0)/len(df_cleaned)*100)

df_cleaned=dfAll.copy()
df_cleaned = dfAll.dropna(subset=['PFCdiff'])
dfAllnrem=df_cleaned[df_cleaned['RatioNREM_REM']>0.5]
dfAllrem=df_cleaned[df_cleaned['RatioNREM_REM']<-0.5]
dfAllns=df_cleaned[df_cleaned['RatioNREM_REM']<=0.5]
dfAllns=dfAllns[dfAllns['RatioNREM_REM']>=-0.5]

print('PFC')
print(sum(dfAllnrem['PFCdiff']>0)/len(df_cleaned)*100)
print(sum(dfAllns['PFCdiff']>0)/len(df_cleaned)*100)
print(sum(dfAllrem['PFCdiff']>0)/len(df_cleaned)*100)

df_cleaned=dfAll.copy()
df_cleaned = dfAll.dropna(subset=['GBLdiff'])
dfAllnrem=df_cleaned[df_cleaned['RatioNREM_REM']>0.5]
dfAllrem=df_cleaned[df_cleaned['RatioNREM_REM']<-0.5]
dfAllns=df_cleaned[df_cleaned['RatioNREM_REM']<=0.5]
dfAllns=dfAllns[dfAllns['RatioNREM_REM']>=-0.5]

print('GBL')
print(sum(dfAllnrem['GBLdiff']>0)/len(df_cleaned)*100)
print(sum(dfAllns['GBLdiff']>0)/len(df_cleaned)*100)
print(sum(dfAllrem['GBLdiff']>0)/len(df_cleaned)*100)

if Nr=='L1':
    dfAll.to_excel(f'C:/Users/Manip2/Documents/ElifePaper/Rawdata/L1_Act_Spdl.xlsx')
else:
    dfAll.to_excel(f'C:/Users/Manip2/Documents/ElifePaper/Rawdata/L2&3_Act_Spdl.xlsx')


print('INACTIVE')

print('S1')
print(sum(dfAllnrem['S1diff']<0)/len(df_cleaned)*100)
print(sum(dfAllns['S1diff']<0)/len(df_cleaned)*100)
print(sum(dfAllrem['S1diff']<0)/len(df_cleaned)*100)

df_cleaned=dfAll.copy()
df_cleaned = dfAll.dropna(subset=['PFCdiff'])
dfAllnrem=df_cleaned[df_cleaned['RatioNREM_REM']>0.5]
dfAllrem=df_cleaned[df_cleaned['RatioNREM_REM']<-0.5]
dfAllns=df_cleaned[df_cleaned['RatioNREM_REM']<=0.5]
dfAllns=dfAllns[dfAllns['RatioNREM_REM']>=-0.5]

print('PFC')
print(sum(dfAllnrem['PFCdiff']<0)/len(df_cleaned)*100)
print(sum(dfAllns['PFCdiff']<0)/len(df_cleaned)*100)
print(sum(dfAllrem['PFCdiff']<0)/len(df_cleaned)*100)

df_cleaned=dfAll.copy()
df_cleaned = dfAll.dropna(subset=['GBLdiff'])
dfAllnrem=df_cleaned[df_cleaned['RatioNREM_REM']>0.5]
dfAllrem=df_cleaned[df_cleaned['RatioNREM_REM']<-0.5]
dfAllns=df_cleaned[df_cleaned['RatioNREM_REM']<=0.5]
dfAllns=dfAllns[dfAllns['RatioNREM_REM']>=-0.5]

print('GBL')
print(sum(dfAllnrem['GBLdiff']<0)/len(df_cleaned)*100)
print(sum(dfAllns['GBLdiff']<0)/len(df_cleaned)*100)
print(sum(dfAllrem['GBLdiff']<0)/len(df_cleaned)*100)


In [ ]:
dfd = dfa[dfa['Drug'] == 'Baseline'] 
dfd = dfd[dfd['GlobalSpindle'] == 'Global'] 

dfd['SpikeActivityPreference'].replace(np.nan, 'NoPref', inplace=True)
dfd['CalciumActivityPreference'].replace(np.nan, 'NoPref', inplace=True)
dff = dfd.pivot_table(index='Unit_ID', columns=[dfd['SpikeActivityPreference']], values='SpikeActivityPreference', aggfunc='count', fill_value=0)
dff['Total']=dff.sum(axis=1)

dff['%Before']=np.round(dff['Before']/dff['Total']*100,2)
dff['%During']=np.round(dff['During']/dff['Total']*100,2)
dff['%After']=np.round(dff['After']/dff['Total']*100,2)
dff['%Response']=np.round((dff['Before']+dff['During']+dff['After'])/dff['Total']*100,2)

bin_edges = range(1,100,2)
plt.close()
plt.hist(dff['%Before'], bins=bin_edges,  alpha=0.5, color='green')
plt.hist(dff['%During'], bins=bin_edges,  alpha=0.5, color='red')
plt.hist(dff['%After'], bins=bin_edges,  alpha=0.5, color='blue')
plt.hist(dff['%Response'], bins=bin_edges,  alpha=0.5, color='black')
print('% Before=',round(np.mean(dff['%Before']),2))
print('% During=',round(np.mean(dff['%During']),2))
print('% After=',round(np.mean(dff['%After']),2))
print('% Response=',round(np.mean(dff['%Response']),2))

plt.show()


In [ ]:
dff[dff['%During']>0]

In [ ]:
dfd = dfa[dfa['Drug'] == 'Baseline'] 
dfd = dfd[dfd['GlobalSpindle'] == 'Local'] 
dfd = dfd[dfd['SpdlStartLocation'] == 'S1'] 

dfd['SpikeActivityPreference'].replace(np.nan, 'NoPref', inplace=True)
dfd['CalciumActivityPreference'].replace(np.nan, 'NoPref', inplace=True)
dff = dfd.pivot_table(index='Unit_ID', columns=[dfd['SpikeActivityPreference']], values='SpikeActivityPreference', aggfunc='count', fill_value=0)
dff['Total']=dff.sum(axis=1)

dff['%Before']=np.round(dff['Before']/dff['Total']*100,2)
dff['%During']=np.round(dff['During']/dff['Total']*100,2)
dff['%After']=np.round(dff['After']/dff['Total']*100,2)
dff['%Response']=np.round((dff['Before']+dff['During']+dff['After'])/dff['Total']*100,2)

dff

bin_edges = range(0,30,1)
plt.close()
plt.hist(dff['%Before'], bins=bin_edges,  alpha=0.5, color='green')
plt.hist(dff['%During'], bins=bin_edges,  alpha=0.5, color='red')
plt.hist(dff['%After'], bins=bin_edges,  alpha=0.5, color='blue')
plt.hist(dff['%Response'], bins=bin_edges,  alpha=0.5, color='black')

print('% Before=',round(np.mean(dff['%Before']),2))
print('% During=',round(np.mean(dff['%During']),2))
print('% After=',round(np.mean(dff['%After']),2))
print('% Response=',round(np.mean(dff['%Response']),2))

plt.show()

In [ ]:
dff[dff['%During']>20]

In [ ]:
dfd = dfa[dfa['Drug'] == 'Baseline'] 
dfd = dfd[dfd['GlobalSpindle'] == 'Local'] 
dfd = dfd[dfd['SpdlStartLocation'] == 'PFC'] 

dfd['SpikeActivityPreference'].replace(np.nan, 'NoPref', inplace=True)
dfd['CalciumActivityPreference'].replace(np.nan, 'NoPref', inplace=True)
dff = dfd.pivot_table(index='Unit_ID', columns=[dfd['SpikeActivityPreference']], values='SpikeActivityPreference', aggfunc='count', fill_value=0)
dff['Total']=dff.sum(axis=1)

dff['%Before']=np.round(dff['Before']/dff['Total']*100,2)
dff['%During']=np.round(dff['During']/dff['Total']*100,2)
dff['%After']=np.round(dff['After']/dff['Total']*100,2)
dff['%Response']=np.round((dff['Before']+dff['During']+dff['After'])/dff['Total']*100,2)

dff

bin_edges = range(1,100,2)
plt.close()
plt.hist(dff['%Before'], bins=bin_edges,  alpha=0.5, color='green')
plt.hist(dff['%During'], bins=bin_edges,  alpha=0.5, color='red')
plt.hist(dff['%After'], bins=bin_edges,  alpha=0.5, color='blue')
plt.hist(dff['%Response'], bins=bin_edges,  alpha=0.5, color='black')

print('% Before=',round(np.mean(dff['%Before']),2))
print('% During=',round(np.mean(dff['%During']),2))
print('% After=',round(np.mean(dff['%After']),2))
print('% Response=',round(np.mean(dff['%Response']),2))
plt.show()

In [ ]:
dff[dff['%During']>20]